**Aplicação de técnicas de classificação de texto na arquitetura de Chatbots hierárquicos**

**Mestrado Profissional em Computação Aplicada - PPCA - Universidade de Brasília**

**Disciplina**: Mineração de Dados

**Orientador**: Prof. Dr. Marcelo Ladeira <mladeira@unb.br>

**Alunos**: 
 - José Ronaldo Agra De Souza Filho <jose.agra@aluno.unb.br>
 - Bruno Gomes Resende <bruno.resende@aluno.unb.br>
 - Célio Castro Wermelinger <celio.wermelinger@aluno.unb.br> 

**Palavras-Chave:**

Chatbot; Text Classification; Machine Learning; Text Mining; NLP;

In [ ]:
# Install packages
# Note: use Python 3.8 env
#!pip install --pre pycaret

In [ ]:
# Import packages
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pycaret.classification import *
import pandas as pd

# download das stopwords para o idioma português
nltk.download('stopwords')

In [ ]:
# Montagem do dataset
df_covid = pd.read_csv('data/exemplos-treinamento-covid.csv', delimiter=';')
df_seloturismo = pd.read_csv('data/exemplos-treinamento-seloturismo.csv', delimiter=';')
df_tuberculose = pd.read_csv('data/exemplos-treinamento-tuberculose.csv', delimiter=';')
df_teste = pd.read_csv('data/dados-testes-experimentos.csv', delimiter=';')

df_all = pd.concat([df_covid, df_seloturismo, df_tuberculose, df_teste], axis=0)
df_all.reset_index(drop=True, inplace=True)
df_all['category'] = df_all.reset_index().category.map({'covid':0, 'seloturismo':1, 'tuberculose':2})

df_all.describe()

In [ ]:
# recupera as stopwords do idioma português
stop_words = stopwords.words('portuguese')

# vetorização das palavras
tv = TfidfVectorizer(lowercase=True, stop_words=stop_words, strip_accents='unicode', token_pattern="[A-Za-z]+")

tf_idf_dados = tv.fit_transform(df_all['input'])

# vetorização para o dataframe referente à categoria COVID
df_dados_vetorizados = pd.DataFrame(tf_idf_dados.toarray(), columns=tv.get_feature_names())
df_dados_vetorizados['target_cat'] = df_all.reset_index().category

dados_treino_validacao = df_dados_vetorizados.sample(frac=0.8, random_state=786).reset_index(drop=True)
dados_teste = df_dados_vetorizados.drop(dados_treino_validacao.index).reset_index(drop=True)

print('Data for Modeling: ' + str(dados_treino_validacao.shape))
print('Unseen Data For Predictions: ' + str(dados_teste.shape))

dados_treino_validacao

In [ ]:
# Setup
# Iniciando os recursos da biblioteca, passando como alvo a coluna 'class' do dataset
setup = setup(data=dados_treino_validacao, target='target_cat', session_id=20221,
              train_size = 0.7, test_data=dados_teste, fold=10, silent=True, fix_imbalance=True)

In [ ]:
metricas = get_metrics()
metricas

In [ ]:
# Gerando modelos com diferentes abordagens e escolhendo o melhor por cross-validation
modelos = compare_models()

In [ ]:
# Imprimindo melhor modelo
print(modelos)

In [ ]:
# Matriz de confusão TF-IDF
modelo = create_model('et')

In [ ]:
# realiza o ensemble dos modelos e seleciona o melhor
melhor_modelo_bagging = ensemble_model(modelo, choose_better=True, return_train_score=True)

In [ ]:
# Imprimindo melhor modelo
print(melhor_modelo_bagging)

In [ ]:
plot_model(melhor_modelo_bagging, plot = 'confusion_matrix')

## Predição

In [ ]:
# Realizando predição com dateset de validação
predict_model(modelo)

In [ ]:
modelo_final = finalize_model(melhor_modelo_bagging)
predicoes = predict_model(modelo_final)

In [ ]:
predicoes.head(200)


In [ ]:
save_model(modelo_final,'models/20220914-modelo-final-multiclasses-tdidf-et')
